In [2]:
library('tidyverse', 'stats')

(a)

In this case, we set up our transition matrix given the poisson distribution with $\lambda = 6/5$. However, we want to set up the matrix with states 0 (no cars) and 6 (cars have to leave) as absorbing; i.e., probability 1. Thus, we have a 7x7 transition matrix.

We know that as time moves towards infinity, the probability of not being absorbed approaches 0, so we can just push the matrix through multiplications of itself in order to clear out the transitions between our absorbing states, leaving us with just the ultimate probabilities of cars emptying and cars turning away.



In [54]:
## a

lambda <- 6/5
p_vec <- matrix(sapply(0:5, function(i) ppois(i,lambda)-ppois(i-1,lambda)),nrow=1)
G <- rbind(c(1, 0, 0, 0, 0, 0, 0), 
           c(p_vec, 1-sum(p_vec)),
           c(0, p_vec[1:5], 1-sum(p_vec[1:5])),
           c(0, 0, p_vec[1:4], 1-sum(p_vec[1:4])),
           c(0, 0, 0, p_vec[1:3], 1-sum(p_vec[1:3])),
           c(0, 0, 0, 0, p_vec[1:2], 1-sum(p_vec[1:2])),
           c(0, 0, 0, 0, 0, 0, 1))

cat('G')
G

G

1.0000000,0.0000000,0.0000000,0.00000000,0.00000000,0.000000000,0.000000000
0.3011942,0.3614331,0.2168598,0.08674393,0.02602318,0.006245563,0.001500225
0.0000000,0.3011942,0.3614331,0.21685983,0.08674393,0.026023180,0.007745788
0.0000000,0.0000000,0.3011942,0.36143305,0.21685983,0.086743933,0.033768968
0.0000000,0.0000000,0.0000000,0.30119421,0.36143305,0.216859833,0.120512901
0.0000000,0.0000000,0.0000000,0.00000000,0.30119421,0.361433054,0.337372734
0.0000000,0.0000000,0.0000000,0.00000000,0.00000000,0.000000000,1.000000000


In [55]:
G_stat <- G
for(i in 1:100){G_stat <- G_stat%*%G_stat}
G_stat

1.00000000,0,0,0,0,0,0.0000000
0.65444723,0,0,0,0,0,0.3455528
0.41729160,0,0,0,0,0,0.5827084
0.25453214,0,0,0,0,0,0.7454679
0.14295438,0,0,0,0,0,0.8570456
0.06742759,0,0,0,0,0,0.9325724
0.00000000,0,0,0,0,0,1.0000000


Since we started with two cars in the lot, the desired probability is that we go into state 0 from state 2: 41.73%

In [56]:
cat('Probability of Gas Station Emptying: ',G_stat[3,1])

Probability of Gas Station Emptying:  0.4172916

(b)

Unlike the 7x7 matrix shown above, for this part of the problem we're not concerned with the probability that cars will get turned away - only the probability that the lot will empty, and thus we will use a 6x6 matrix to model the expected time until the lot empties.

In [57]:
new_vec <- matrix(c(G[2,1:5], 1-sum(G[2,1:5])), nrow=1)
new_G <- rbind(new_vec, new_vec, 
               c(0, new_vec[1:4], 1-sum(new_vec[1:4])),
               c(0, 0, new_vec[1:3], 1-sum(new_vec[1:3])),
               c(0, 0, 0, new_vec[1:2], 1-sum(new_vec[1:2])),
               c(0, 0, 0, 0, new_vec[1], 1-sum(new_vec[1])))
cat('New G')
new_G

New G

0.3011942,0.3614331,0.2168598,0.08674393,0.02602318,0.007745788
0.3011942,0.3614331,0.2168598,0.08674393,0.02602318,0.007745788
0.0000000,0.3011942,0.3614331,0.21685983,0.08674393,0.033768968
0.0000000,0.0000000,0.3011942,0.36143305,0.21685983,0.120512901
0.0000000,0.0000000,0.0000000,0.30119421,0.36143305,0.337372734
0.0000000,0.0000000,0.0000000,0.00000000,0.30119421,0.698805788


my probabilities kept getting truncated in R for some reason so I had to switch to python

In [1]:
import numpy as np

In [2]:
B=np.array([
        [0.301194212,0.301194212,0,0,0,0],
        [0.361433054,0.361433054,0.301194212,0,0,0],
        [0.216859833,0.216859833,0.361433054,0.301194212,0,0],
        [0.086743933,0.086743933,0.216859833,0.361433054,0.301194212,0],
        [0.02602318,0.02602318,0.086743933,0.216859833,0.361433054,0.301194212],
        [0.007745788,0.007745788,0.033768968,0.120512901,0.337372734,0.698805788]
        ])


exp=0.0
for i in range(1,1000):
    prob=np.array([0,0,1,0,0,0]).dot(B.T)[0]
    exp+=i*prob
    B=B.dot(B)
print('Expected intervals: {}'.format(exp))

Expected intervals: 51.70805830862676


The expected amount of time intervals before the system empties is 52, or 8 hours and 40 minutes.

(a)

We can say that a state is feasible if it's in $C = (a, b) : 0 \leq a, b \leq 4,(a, b) \neq (4, 4)$

Given that a state is feasible such that $(a,b) \in C$, we can say:

If $(a,b) \leq 3:$

$\begin{equation}
  P_{(a,b),(c,d)}=\begin{cases}
    p, & \text{if $c = a + 1, d = b$}.\\
    1-p, & \text{if $c = a, d = b + 1$}.\\
    0 & \text{otherwise}.
  \end{cases}
\end{equation}$

If $a\ or\ b = 4:$

$\begin{equation}
  P_{(a,b),(c,d)}=\begin{cases}
    1, & \text{if $c,d = a,b$}.\\
    0 & \text{otherwise}.
  \end{cases}
\end{equation}$

(b)

This is just a first transition analysis.

If $a, b \leq 4:$

$p_{(a,b)} = p × p_{(a+1,b)} + (1 − p) × p_{(a,b+1)}, \forall \ (a, b) \in C$

Where we have a boundary that says 

$\forall {(a, b) \in C : a = 4\ or\ b = 4}$, such that $p_{(a,b)} = 0$ except when $p_{(1,4)} = p_{(4,1)} = 1$

(c)

It's possible to use a smaller state space. If we consider the possibility that both teams have each won 2 games, then we can basically say that it's impossible for either team to have won the series in 5 games. This is another boundary such that $p_{(3,2)} = p_{(2,3)} = 0$ with $a + b > 5$

(d)

This is another first transition analysis exercise.

We can let $T =$ time left in the series. The system of equations is as follows.

$T_{(a,b)} = 1 + p × T_{(a+1,b)} + (1 − p) × T_{(a,b+1)}$ solving for $T_{(0,0)}$

If $a\ or\ b = 4$ then the series is over already. These are absorbing states. Thus, expected length after this point is 0: $T_{(a,b)}=0$



(e)

This just means that you you need to keep track of an extra step where the state has your $(a,b)$, but also keeps track of the last team to win - call it $l$. This variable can only take 2 values though - whichever team won in the last round.

In [1]:
morning <- matrix(rbind(c(.04, .41, .55), c(.36, .21, .43), c(.16, .28, .56)), nrow=3,ncol=3)
afternoon <- matrix(rbind(c(.26, .37, .37), c(.09, .21, .70), c(.23, .28, .49)), nrow=3,ncol=3)
evening <- matrix(rbind(c(.38, .37, .25), c(.49, .06, .45), c(.16, .22, .62)), nrow=3,ncol=3)
night <- matrix(rbind(c(.15, .13, .72), c(.62, .02, .36), c(.10, .63, .27)), nrow=3,ncol=3)

cat('morning')
morning
cat('afternoon')
afternoon
cat('evening')
evening
cat('night')
night

morning

0.04,0.41,0.55
0.36,0.21,0.43
0.16,0.28,0.56


afternoon

0.26,0.37,0.37
0.09,0.21,0.70
0.23,0.28,0.49


evening

0.38,0.37,0.25
0.49,0.06,0.45
0.16,0.22,0.62


night

0.15,0.13,0.72
0.62,0.02,0.36
0.10,0.63,0.27


(a) 

X is not a time homogenous markov chain. The transition matrices vary based on the time of day, incorporating a seasonality effect on the the probabilities of moving between low, medium, and high workloads.

We can find the transition probabilities of X by simple matrix multiplication of each shift, starting with the shift you begin in and ending with the shift before the one you end in. For example, if we wanted to compute the probability distributions for starting in the afternoon and ending in the afternoon, we would do:

$X = P_{afternoon}*P_{evening}*P_{night}*P_{morning}$

In [2]:
X_n = afternoon%*%evening%*%night%*%morning
cat('X_n')
X_n

X_n

0.2011911,0.2850189,0.5137900
0.2107338,0.2812866,0.5079795
0.2032133,0.2846044,0.5121823


(b)

Because $X_0$ is $P_{afternoon}$, we know that as $Y_m$ moves forward 1, $X_{4m}$ moves forward 4, and we're back at the afternoon matrix. Therefore, $Y_m$ is simply the afternoon matrix, and is actually a time homogenous markov chain.


In [3]:
#b

Y_m <- afternoon
cat('Ym')
Y_m

Ym

0.26,0.37,0.37
0.09,0.21,0.70
0.23,0.28,0.49


(c)

Here we can take the initial distribution $\mu$ from the afternoon of 1/7/2019 as being a high workload.

From here we step forward from afternoon in state 3 > evening > night > morning > afternoon, 4 steps in total. Once we are in afternoon of the next day, we can determine the probability that the workload will be high as 52.53%

In [5]:
x_c <- afternoon[3,]%*%evening%*%night%*%morning%*%afternoon
x_c[3]

[1] 0.5253813